In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

# from simulator import Simulation
from tqdm.auto import tqdm
import itertools
import matplotlib as mpl
from collections import OrderedDict, defaultdict

from voting_mechanism_design.agents.pairwise_badgeholder import PairwiseBadgeholder, PairwiseBadgeholderPopulation
from voting_mechanism_design.projects.project import Project, ProjectPopulation
from voting_mechanism_design.funds_distribution.pairwise_binary import PairwiseBinary
from voting_mechanism_design.sim import RoundSimulation

from voting_mechanism_design.agents.quorum_badgeholder import QuorumBadgeholder, QuorumBadgeholderPopulation
from voting_mechanism_design.funds_distribution.threshold_and_aggregate import ThresholdAndAggregate

import itertools
import numpy as np
import choix
import math

import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

from tqdm.auto import tqdm
from joblib import Parallel, delayed

warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)

In [3]:
def run_single_quorum_threshold_simulation_v2(
        n_badgeholders=50,
        badgeholder_expertise_vec=None,
        badgeholder_laziness_vec=None,
        n_projects=100,
        project_impact_vec=None,
        # configuration specific to quorum+threshold voting method
        quorum=5,
        scoring_fn='mean',
        min_vote_amt=1,
        max_vote_amt=16,
        max_funding=100,
        coi_factor=0.0,
        coi_project_ix=None,
        coi_badgeholder_ix=None,
        random_seed=1234
    ):
    # create voters
    voters = []
    for ii in range(n_badgeholders):
        assert badgeholder_laziness_vec[ii] >= 0

        if ii == coi_badgeholder_ix:
            coi_factor_bh = coi_factor
            coi_project_ix = coi_project_ix
        else:
            coi_factor_bh = 0.0
            coi_project_ix = None

        v = QuorumBadgeholder(
            badgeholder_id=ii,
            total_funds=max_funding,
            min_vote=min_vote_amt,  
            max_vote=max_vote_amt,
            laziness=badgeholder_laziness_vec[ii],
            expertise=badgeholder_expertise_vec[ii],
            coi_factor=coi_factor_bh,
            coi_project_ix_vec=coi_project_ix,
        )
        voters.append(v)
    badgeholder_pop=QuorumBadgeholderPopulation()
    badgeholder_pop.add_badgeholders(voters)

    # add projects
    projects = []
    for ii in range(n_projects):
        p = Project(
            project_id=ii,
            true_impact=project_impact_vec[ii],  
            owner_id=None,  
        )
        projects.append(p)
    project_population = ProjectPopulation()
    project_population.add_projects(projects)

    # ##  Voting Simulation
    fund_distribution_model = ThresholdAndAggregate(
        scoring_method=scoring_fn,
        quorum=quorum,
        min_amount=0
    )
    simulation_obj = RoundSimulation(
        badgeholder_population=badgeholder_pop,
        projects=project_population,
        funding_design=fund_distribution_model,
        random_seed=random_seed
    )
    projects = project_population.get_projects()
    simulation_obj.run()

    projectid2score = simulation_obj.projectid2score

    # for easy comparison
    projectid2score = OrderedDict(projectid2score)
    ranks = np.argsort(np.asarray(list(projectid2score.values())))
    projectid2rank = {ii:ranks[ii] for ii in range(len(ranks))}
    
    return projectid2rank, ranks, simulation_obj, projectid2score

In [13]:
# test 1
n_badgeholders=50
n_projects=100
max_funding=100
random_seed=1234
badgeholder_expertise_vec = 0.75*np.ones(n_badgeholders)
badgeholder_laziness_vec = 0.0*np.ones(n_badgeholders)
project_impact_vec = np.random.uniform(0,1,n_projects)
quorum=5
scoring_fn='mean'
min_vote_amt=1
max_vote_amt=16
max_funding=100

rs = 112

# no COI
_, ranks_no_coi, _, _ = run_single_quorum_threshold_simulation_v2(
    n_badgeholders=n_badgeholders,
    badgeholder_expertise_vec=badgeholder_expertise_vec,
    badgeholder_laziness_vec=badgeholder_laziness_vec,
    n_projects=n_projects,
    project_impact_vec=project_impact_vec,
    # configuration specific to quorum+threshold voting method
    quorum=quorum,
    scoring_fn=scoring_fn,
    min_vote_amt=min_vote_amt,
    max_vote_amt=max_vote_amt,
    max_funding=max_funding,
    random_seed=rs
)
print('Done 1')

# with COI
coi_factor = 0.75
coi_project_ix = [n_projects-1]
badgeholder_coi_ix = 0
_, ranks_coi, _, _ = run_single_quorum_threshold_simulation_v2(
    n_badgeholders=n_badgeholders,
    badgeholder_expertise_vec=badgeholder_expertise_vec,
    badgeholder_laziness_vec=badgeholder_laziness_vec,
    n_projects=n_projects,
    project_impact_vec=project_impact_vec,
    # configuration specific to quorum+threshold voting method
    quorum=quorum,
    scoring_fn=scoring_fn,
    min_vote_amt=min_vote_amt,
    max_vote_amt=max_vote_amt,
    max_funding=max_funding,
    random_seed=rs,
    coi_factor=coi_factor,
    coi_project_ix=coi_project_ix,
    coi_badgeholder_ix=badgeholder_coi_ix
)

In [14]:
np.where(ranks_no_coi == coi_project_ix[0])

(array([46]),)

In [15]:
np.where(ranks_coi == coi_project_ix[0])

(array([46]),)